#  GPT-2 Generation and Fine-Tuning

This notebook explores GPT-2 (Generative Pretrained Transformer-2) from OpenAI. Read more about it [here](https://openai.com/blog/better-language-models/).

Activities include:

0. Setup
1. Generate samples from pre-trained gpt-2model
2. Fine-tune gpt-2 on text of your choosing. 

Adapted by Robert Twomey (rtwomey@unl.edu) for Machine Learning for the Arts SP22 from this [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](http://minimaxir.com). See his repo [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple).

# 0. Setup

Run once to install the library

In [1]:
%pip install -q gpt-2-simple

Note: you may need to restart the kernel to use updated packages.


restart the kernel and run the imports

In [2]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [3]:
!nvidia-smi

Mon Feb 21 15:32:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

note the memory usage (0MiB / 32510MiB) for the Tesla V100.
You can re-rerun the above cell to see what memory your code/models are using during training.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the the current working directory at `/models/<model_name>`.

In [4]:
model_name = "355M" # largest model we can fine-tune

run once to download the file

In [5]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 336Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.17Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 995Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [01:05, 21.7Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.35Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.77Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 5.43Mit/s]                                                      


# 1. Generate Text From The Pretrained Model

If you want to generate text from the pretrained model pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`. (This is currently the only way to generate text from the 774M or 1558M models with this notebook.)

In [6]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

2022-02-21 15:34:12.493053: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 15:34:13.117623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


## Sample from the model
The follow cell samples from gpt-2, using the provided prefix (seed) and other parameters. It starts the TF session and generates the samples.

Try changing the parameters below to change the output: 
- `prefix` is the prompt. This will be the starting string/seed for your generation. Use your own text. 
- `temperature` sets the variability/randomness of the output. Range 0.0-1.0
- `length` sets the lenght of output (in tokens). max is 1024.


In [7]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="Kansas City, Missouri",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Kansas City, Missouri)

26. Buffalo, New York (Buffalo, New York)

27. Columbus, Ohio (Columbus, Ohio)

28. San Diego, California (San Diego, California)

29. Richmond, Virginia (Richmond, Virginia)

30. Wichita Falls, Texas (Wichita Falls, Texas)

31. New Orleans, Louisiana (New Orleans, Louisiana)

32. Los Angeles, California (Los Angeles, California)

Kansas City, Missouri

Name: M.J. Wilson

Position: Defensive end

Height: 6-2

Weight: 238

Age: 22

College: Alabama

Round(s): 7

Drafted: 2012, No. 100 overall, by the New England Patriots

The Patriots signed him to a four-year contract worth $12 million. But Wilson didn't play much in 2013, as he missed four games with a torn ACL. He was inactive
Kansas City, Missouri, USA

September 16, 2014


A juvenile male was observed with a large white or dark patch on his upper left arm. His arm was extremely large and pointed. The patch was a little large and had a couple of small dark spots on the outside. The patch was visible from abov

## Activities
- try varying the prefix. 
  - what length of prefix works best with the given model? 
  - how does the choice of prefix change the format/form of the output.
- try varying the temperature.
- try loading the different sized models (124M, 355M, 774M, 1558M) and generate text without changing the other parameters. 
  - Do you notice any qualitative differences in the output? 

# 2. Fine-Tuning GPT-2

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [8]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-21 15:35:31.227388: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


## Upload a text file
For this, we will use a text file you provide to finetune (continue training) GPT-2. You can use any plain text (.txt) file. 

Simply drag and dropy our text file into the file browser at left. 

Once you have uploaded your file, update the file name in the cell below, then run it.

In [9]:
file_name = "UFO sightings.rtf" # your file here

## Run the finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every `save_every` steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them. If your input text is smaller, training might proceed more quickly.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [10]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=150,
              restore_from='fresh', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=70,
              save_every=150
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1



2022-02-21 15:35:42.842697: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  5.60it/s]

dataset has 40695 tokens
Training...


[10 | 29.32] loss=1.97 avg=1.97
[20 | 49.26] loss=1.80 avg=1.88
[30 | 69.18] loss=1.69 avg=1.82
[40 | 89.10] loss=1.65 avg=1.78
[50 | 109.03] loss=1.61 avg=1.74
[60 | 128.94] loss=1.52 avg=1.70
[70 | 148.86] loss=1.49 avg=1.67
======== SAMPLE 1 ========
 a more general, or even generalizing, meaning. For "pardon," the Oxford English Dictionary says "to be granted and allow to be granted or allowed to be granted" but then cites two other Oxford dictionaries, the British and American. There is one more Oxford dictionary, too: the Oxford New International Dictionary, and it has no suggestion of using the general usage of all dictionaries. Perhaps they will give it a closer look in time.

The first mention of "hearing" in the New New York Times and elsewhere comes in a letter, Oct. 9, to the correspondent of the New York Daily News. It is from a lawyer in the firm of H. N. T. Gorman, and it is from Mr. Gorman to the reporter:

"In the old days before the State was so organized, the State D

## Notes on finetuning

Keep an eye on the loss, and how quickly it is dropping. A too-rapid drop in loss could be a sign of overfitting, and a learning rate (lr) that is too high. 

After the model is trained, you can download the checkpoint folder to save your work. Training checkpoints are saved to `checkpoint/run1` (or whatever you chose for the run name above).

You can compress it to a rar file and download that. Ask the instructor how.

You're done! Feel free to go to the Generate Text From The Trained Model section to generate text based on your retrained model.

## Finetune some more

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [ ]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

To fine-tune some more, run the following. Be sure to increase the number of steps (if it was `500` before, change to `1000` to train for 500 more. the number is cumulative).

In [ ]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=300,
              restore_from='latest', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=100,
              save_every=100
              )

# 3. Generate Text From The Finetuned Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
# gpt2.generate(sess, run_name='run1') # no prefix, unconditional generation

In [12]:
gpt2.generate(sess, run_name='run1', prefix="March, 23rd 2002")

March, 23rd 2002. 2.45 pm. 10 miles from runway, 5,000 feet. A loud whizzing sound was heard for 10 seconds. No further details in files. Witness: R.T. Washburn, flying a Cessna 172.


5 July 2002; Goose Bay, Labrador. 1.35 am. Witnesses: 1 pilot, 1 radar observer. Two objects, about the size of a pick-up truck, flew from north to south, with a tail thrust, for about 1 hour.


29 April 2002; Crater Lake, Michigan. 11:35 p.m. Witness: N/A. One round, bright-green object flew erratically for 20 minutes, veered, and made an emergency landing at the bottom of the lake. ____________________


28 May 2002; near Laredo, Texas. 8:30 p.m. Witness: ************. One object, as big as the Saturn V, flew straight and level for a few seconds, then gave an abrupt jerky, jerky climb. ____________________


23 June 2002; Richmond, Virginia. 6:30 a.m. Witness: ************. Three flat, star-like objects, with tops like dominoes, flew very fast, jerky, erratically, through the air for 3-4 minutes. _____

## Notes
If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.85,
              prefix="Kansas City, Missouri"
              nsamples=5,
              batch_size=5
              )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

Download the file by hand in the browser at left.

## Load a Trained Model Checkpoint

Uploaded your saved checkpoint and unzip it.

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

This will reset or start the tensorflow session as necessary.

In [ ]:
tf.compat.v1.reset_default_graph()

if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

gpt2.load_gpt2(sess, run_name='run1')

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# License

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Reference
- Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
- Original repo: [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by [Max Woolf](http://minimaxir.com). 
- Original [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) from Max.